In [1]:
import requests
from datetime import datetime
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import yaml
import ast
import glob
import numpy as np
import psycopg2
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int32, psycopg2._psycopg.AsIs)
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

In [2]:
# Время ожидания между запросами, в сек.
wait = 0.3

# Функции подключения к PostgreSQL.
param_dic = {
    'host'      : '192.168.200.12',
    'database'  : 'hh_pet',
    'user'      : 'hh_user',
    'password'  : 'DWuh1289h12ej2h32178dh2-38hD'
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    print("Connection successful")
    return conn

def execute_mogrify_insert(conn, df, table):
    """
    Using cursor.mogrify() to build the bulk insert query
    then cursor.execute() to execute the query
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    cursor = conn.cursor()
    
    col = ''
    for i in range(0, len(df.columns)):
        col += '%s,'
    values = [cursor.mogrify("(" + col[:-1] + ")", tup).decode('utf8') for tup in tuples]
    query  = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    
    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_mogrify() done")
    cursor.close()

In [ ]:
# Итоговый список для создания вакансий. Будущий тип: список словарей.
dict_prof = []

# Составим список всех категорий.
url = "https://www.profguide.io/professions/"
response = requests.get(url)

pattern = re.compile('<li><a href="/professions/(.*)" title=".*, профессии"')
category_sites = pattern.findall(response.text)

pattern = re.compile('<li><a href="/professions/category/.*/" title=".*профессии">(.*)</a> <span>.*</span> </li>')
category_names = pattern.findall(response.text)

# По каждой категории составляем список всех вакансий.
for cat in range(0, len(category_names)):
    response = requests.get(url + category_sites[cat])

    # Узнаём сколько записей.
    pattern = re.compile('<div class="summary">Показаны записи <b>.*</b> из <b>(.*)</b>.</div><div class="row tile-items-list">')
    amount = int(pattern.findall(response.text)[0])

    # Составляем список всех вакансий для категории.
    professions = []
    page = 1
    while len(professions) < amount:
        response = requests.get(url + category_sites[cat] + '?page=' + str(page))
        pattern = re.compile('<a class="nl" href="/professions/(.*\.html)" title="')
        professions += pattern.findall(response.text)
        page += 1
        time.sleep(wait)

    # Теперь получаем данные по каждой профессии.
    for pro in professions:
        # Загружаем страницу с профессией.
        response = requests.get(url + pro)
        
        # Запоминаем название.
        pattern = re.compile('<meta property="og:title" content="(.*)">')
        prof_name = pattern.findall(response.text)[0]

        # Достаем краткое описание, мало ли понадобится.
        pattern = re.compile('<div class="article-cover__subtitle">(.*)</div>')
        try:
            prof_info = pattern.findall(response.text)[0]
        except:
            pattern = re.compile('<p class="lead">(.*)</p>')
            prof_info = pattern.findall(response.text)[0]
        if not prof_info:
            pattern = re.compile('<meta property="og:description" content="(.*)>')
            prof_info = pattern.findall(response.text)[0]
        prof_info = BeautifulSoup(prof_info, "lxml").text # Очищение текста от тегов.
        
        dict_prof.append({'category': category_names[cat].strip(),
                      'profession': prof_name.strip(),
                      'profession_info': prof_info.strip(),
                      'profession_link': url + pro
                     })
        time.sleep(wait)

# Создадим датафрейм с данными по вакансиям.
df_professions = pd.DataFrame(dict_prof)

Немного заострю внимания на аналитиках: на hh.ru в поиске очень мало результатов для "big data analyst (аналитик данных)", поэтому его заменим на "аналитика данных". Далее вместо "бизнес-аналитик (системный аналитик)" сделаем просто "бизнес-аналитик", т.к. системный отдельно имеется.

In [ ]:
df_professions[df_professions['profession'].str.contains('Аналитик данных')]

In [ ]:
df_professions[df_professions['profession'].str.contains('Бизнес-аналитик')]

In [ ]:
df_professions.loc[100, 'profession'] = 'Аналитик данных'
df_professions.loc[169, 'profession'] = 'Бизнес-аналитик'

In [16]:
df_professions[df_professions['profession'].str.lower().str.contains('программист')]

,category,profession,profession_info,profession_link
112,Информационные технологии (IT),Инженер-программист,Инженер-программист – увлекательная техническа...,https://www.profguide.io/professions/inzhener-...
124,Информационные технологии (IT),Программист Kotlin,Программисты Kotlin занимаются разработкой And...,https://www.profguide.io/professions/kotlin_de...
125,Информационные технологии (IT),Программист JavaScript,"Программист JavaScript – профессия, идеально п...",https://www.profguide.io/professions/javascrip...
126,Информационные технологии (IT),Программист C++,Программистом C++ могут стать юноши и девушки ...,https://www.profguide.io/professions/programmi...
130,Информационные технологии (IT),Программист Swift,"Программист Swift разрабатывает, исправляет, р...",https://www.profguide.io/professions/swift_pro...
131,Информационные технологии (IT),Программист Python,Работа программиста Python — это написание код...,https://www.profguide.io/professions/python_pr...
133,Информационные технологии (IT),Программист C#,Язык программирования C# считается простым в и...,https://www.profguide.io/professions/programme...
136,Информационные технологии (IT),Программист Java,Программирование на Java развивается еще с кон...,https://www.profguide.io/professions/java_prog...
137,Информационные технологии (IT),Программист Ruby,"Язык Ruby достаточно сложный, а опытных разраб...",https://www.profguide.io/professions/ruby_prog...
138,Информационные технологии (IT),Программист PHP,"Специалист, программирующий на языке PHP долже...",https://www.profguide.io/professions/php_devel...


In [17]:
# Добавим программиста .Net для разнообразия.
df_professions.loc[len(df_professions)] = ['Информационные технологии (IT)',
                                           'Программист .Net',
                                           '.NET-разработчик – это программист, основными инструментами в работе которого являются платформа и технологии .NET. Сам по себе .NET представляет огромный набор инструментов, которые связаны друг с другом, позволяющих разработчику трудиться, используя язык программирования С#. Создана платформа.NET была Microsoft, и с самого момента создания этой платформы компания старается активно развивать это свое детище.',
                                           'https://nsk.itstep.org/blog/Kto-takoj-NET-razrabotcik_'
                                          ]
df_professions[df_professions['profession'].str.lower().str.contains('программист')].tail()

,category,profession,profession_info,profession_link
166,Информационные технологии (IT),Программист 1C,"Большинство компаний, использующих эту систему...",https://www.profguide.io/professions/programmi...
170,Информационные технологии (IT),Веб-программист,Веб-программист востребованная и уже довольно ...,https://www.profguide.io/professions/web_progr...
171,Информационные технологии (IT),Программист,Программисты в настоящее время очень востребов...,https://www.profguide.io/professions/programme...
174,Информационные технологии (IT),ERP-программист,"Помимо разработки и обслуживания системы, ERP-...",https://www.profguide.io/professions/erp_progr...
919,Информационные технологии (IT),Программист .Net,".NET-разработчик – это программист, основными ...",https://nsk.itstep.org/blog/Kto-takoj-NET-razr...


In [18]:
# Уберём двойные плюсы чтобы были все языки C в поиске.
# Уберём %, т.к. из-за них почему-то не загружаются данные в БД.
df_professions.loc[:,'profession'] = df_professions['profession'].apply(lambda x: x.replace('++',''))
df_professions.loc[:,'profession_info'] = df_professions['profession_info'].apply(lambda x: x.replace('++',''))
df_professions.loc[:,'profession_info'] = df_professions['profession_info'].apply(lambda x: x.replace('%',' процентов'))

In [19]:
# В собранном списке есть слищком общие профессии, например просто Инженер.
# Подобные вакансии уберем из датафрейма и посмотрим что еще можно увидеть.
df_professions_lower = df_professions['profession'].str.lower().to_frame()

def count_similar_prof(x):
    try:
        tmp = df_professions_lower[(df_professions_lower['profession'] != x) & (df_professions_lower['profession'].str.contains(x))]
        return pd.Series([len(tmp), " | ".join(tmp['profession'].to_list())])
    except:
        return pd.Series([0, 0])
    
df_professions_lower_grp = df_professions_lower.groupby('profession').count().reset_index()
df_professions_lower_grp[['cnt', 'similars']] = df_professions_lower_grp['profession'].apply(count_similar_prof)

df_professions_lower_grp = df_professions_lower_grp[df_professions_lower_grp['cnt'] > 0].sort_values('cnt', ascending=False)
df_professions_lower_grp.head(10)

C:\Users\PHOBOS\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,profession,cnt,similars
591,программист,16,инженер-программист | программист kotlin | про...
443,маркетолог,15,event-маркетолог | influence-маркетолог | марк...
89,аналитик,12,гейм-аналитик | аналитик данных | продуктовый ...
381,консул,11,генетический консультант | erp-консультант | а...
110,архитектор,8,архитектор интеллектуальных систем управления ...
736,стилист,8,техно-стилист | nail-стилист | персональный ст...
620,психолог,7,клинический психолог | патопсихолог | веб-псих...
268,журналист,7,журналист-международник | журналист кино | жур...
636,редактор,7,редактор кино и тв | бьюти-редактор (beauty ed...
640,режиссер,6,режиссер анимационного кино | ассистент режисс...


In [21]:
# Я удалю следующие профессии как общие:
exclude_prof = ['Хирург', 'Дизайнер', 'Художник', 'Инженер']
df_professions = df_professions[~df_professions['profession'].isin(exclude_prof)]

In [22]:
exclude_prof = ['хирург', 'дизайнер', 'художник', 'инженер']
df_professions_lower_grp = df_professions_lower_grp[~df_professions_lower_grp['profession'].isin(exclude_prof)]

Составим поисковую фразу для hh.ru, причем если профессия имеет 1 и более совпадений, то для них сделаю исключения в названии чтобы разделить вакансии, например:
"Маркетолог" и "Event-маркетолог", соответственно поисковая фраза для Маркетолога будет "Маркетолог NOT Event".

In [23]:
df_professions_lower_grp

,profession,cnt,similars
591,программист,16,инженер-программист | программист kotlin | про...
443,маркетолог,15,event-маркетолог | influence-маркетолог | марк...
89,аналитик,12,гейм-аналитик | аналитик данных | продуктовый ...
381,консул,11,генетический консультант | erp-консультант | а...
110,архитектор,8,архитектор интеллектуальных систем управления ...
...,...,...,...
372,колорист,1,колорист автоэмалей
339,испытатель,1,лётчик-испытатель (лётчик экспериментальной ав...
254,дипломат,1,дипкурьер (дипломатический курьер)
231,декоратор,1,текстильный декоратор


In [25]:
df_professions_lower_grp.loc[591].similars

'инженер-программист | программист kotlin | программист javascript | программист c | программист swift | программист python | программист c# | программист java | программист ruby | программист php | embedded-программист | системный программист | программист 1c | веб-программист | erp-программист | программист .net'

In [26]:
def make_query(data):
    query = data.lower()
    tmp = df_professions_lower_grp[df_professions_lower_grp['profession'] == query]
    if len(tmp) > 0:
        similars = tmp.iloc[0,2]
        replaced = re.sub('[–]',' ',similars)
        replaced = re.sub('[-]',' ',replaced)
        replaced = re.sub(query,'',replaced)
        replaced = re.sub('[\|(),]','',replaced)
        replaced = re.sub('\s(он)\s',' ',replaced)
        replaced = re.sub('\s(же)\s',' ',replaced)
        replaced = re.sub('\s(по)\s',' ',replaced)
        replaced = re.sub('\s(\w)\s',' ',replaced)
        replaced = re.sub('\s{2,}',' ',replaced)
        replaced = replaced.strip()
        for word in replaced.split(' '):
            query += ' NOT ' + word.strip()
        return query
    else:
        return query

df_professions['query'] = df_professions['profession'].apply(make_query)

In [27]:
# Сохраню выгрузку.
df_professions.to_csv('df_professions.csv')

In [49]:
# Тестовый прогон. Посчитаем количество найденных вакансий для каждой профессии.
url = 'https://api.hh.ru/vacancies'
def amount_vacancies(query):
    par = {'text': query, 'User-Agent': 'Pet project (some-mail@yandex.ru)', 'search_field':'name', 'per_page':'10', 'page':0}
    response = requests.get(url, params=par).json()
    added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    pages = response['pages']
    found = response['found']
    time.sleep(0.1)
    return found
    
df_professions['amount'] = df_professions['query'].apply(amount_vacancies)

In [50]:
print('Всего вакансий найдено:', df_professions.amount.sum())
df_professions.groupby('category')['amount'].sum().sort_values(ascending=False)

Всего вакансий найдено: 577497


category
Торговля                                                      166104
Менеджмент (управление)                                        78599
Информационные технологии (IT)                                 66320
Финансы, бухгалтерия                                           39756
Медицина                                                       22078
Сервис и туризм                                                18979
Тяжёлая промышленность                                         17995
Секретариат, делопроизводство, АХО                             16930
Строительство и архитектура                                    16114
Силовые структуры                                              15529
Юриспруденция                                                  14789
Маркетинг, реклама и PR                                        13512
Кулинария, пищевая промышленность                              13193
Лёгкая промышленность                                          10949
Инженерное дело          

In [51]:
df_professions.amount.describe()

count      919.000000
mean       628.397171
std       3154.205478
min          0.000000
25%          0.000000
50%          9.000000
75%        147.000000
max      65172.000000
Name: amount, dtype: float64

Вакансий к загрузке для анализа вроде и достаточно, но можно увидеть, что половина всех профессий имеет лишь 7 вакансий - как-то маловато. Чтобы исправить эту ситуацию для тех профессий где менее 10 вакансий на hh.ru изменим способ поиска - включим глобальный поиск.

In [52]:
# Сохраню выгрузку чтобы в последствии не загружать.
df_professions.to_csv('df_professions_first_load.csv')
# Добавить столбец, где будем отмечать профессии по которым необходимо будет делать поиск не только по названию вакансии,
# а по телу вакансии в том числе.
df_professions['global_search'] = False

In [75]:
df_professions = pd.read_csv('df_professions_first_load.csv')

In [53]:
# Второй прогон. Посчитаем количество найденных вакансий для каждой профессии с поиском по вакансии целиком, а не только в названии.
url = 'https://api.hh.ru/vacancies'
def amount_vacancies_global(query):
    if query[1] < 10:
        par = {'text': query[0], 'User-Agent': 'Pet project (some-mail@yandex.ru)', 'per_page':'10', 'page':0}
        response = requests.get(url, params=par).json()
        added_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        pages = response['pages']
        found = response['found']
        time.sleep(0.1)
        return pd.Series([found, True])
    else:
        return pd.Series([query[1], False])

df_professions[['amount_2', 'global_search']] = df_professions[['query', 'amount']].apply(amount_vacancies_global, axis=1)

In [56]:
df_professions.describe()

,amount,amount_2
count,919.000000,919.000000
mean,628.397171,939.425462
std,3154.205478,9023.926028
min,0.000000,0.000000
25%,0.000000,4.000000
50%,9.000000,33.000000
75%,147.000000,228.500000
max,65172.000000,256991.000000


In [57]:
print('Всего вакансий найдено:', df_professions.amount_2.sum())
df_professions.groupby('category')['amount_2'].sum().sort_values(ascending=False)

Всего вакансий найдено: 863332


category
Сервис и туризм                                               276138
Торговля                                                      166106
Менеджмент (управление)                                        79316
Информационные технологии (IT)                                 71097
Финансы, бухгалтерия                                           40664
Медицина                                                       25058
Тяжёлая промышленность                                         18006
Секретариат, делопроизводство, АХО                             16993
Строительство и архитектура                                    16330
Маркетинг, реклама и PR                                        15621
Силовые структуры                                              15542
Юриспруденция                                                  14963
Педагогика и психология                                        14807
Кулинария, пищевая промышленность                              13337
Инженерное дело          

Что можно сказать по результатам второй загрузки (поиск по всей вакансии целиком) вроде бы стало лучше, но если присмотреться к результатам, то слишком много ошибок возникает, например: если поискать по вакансии целиком, а не только в названии такую редкую профессию как "Бондарь", то будет более 30 результатов, а на самом деле в hh.ru есть одна вакансия. Откуда взяли остальные: совпадение слова с адресом места работы, совпадение слова с Фамилией ИП работодателя и т.п.
В общем, оставляем только первый вариант: поиск вакансий только по названию.

In [30]:
df_professions.reset_index(level=0, inplace=True)
df_professions.rename(columns={'index': 'profession_id'}, inplace=True)
df_professions.loc[:,'profession_id'] = df_professions['profession_id'] + 1

In [31]:
df_professions.head()

,profession_id,category,profession,profession_info,profession_link,query
0,1,Медицина,Реабилитолог,Реабилитолог – врач с высшим медицинским образ...,https://www.profguide.io/professions/reabilito...,реабилитолог
1,2,Медицина,Врач,Врач – интересная и социально значимая професс...,https://www.profguide.io/professions/vrach.html,врач NOT клинической NOT лабораторной NOT диаг...
2,3,Медицина,Эмбриолог,"Эмбриолог – специалист, который является ключе...",https://www.profguide.io/professions/ehmbriolo...,эмбриолог
3,4,Медицина,Эфферентолог,"Эфферентолог – специалист, который очищает кро...",https://www.profguide.io/professions/ehfferent...,эфферентолог
4,5,Медицина,Подолог,"Подолог – специалист, который выполняет аппара...",https://www.profguide.io/professions/podolog.html,подолог


In [32]:
df_professions.loc[:,'profession_info'] = df_professions['profession_info'].str.replace('%', ' проц.', regex=False)

In [33]:
conn = connect(param_dic)
execute_mogrify_insert(conn, df_professions, 'professions')
conn.close()

Connection successful
execute_mogrify() done
